# 🔊 Module 8: AI + Hardware Integration

Today, you will give your AI a voice. You will connect your AI's digital detections to a physical device, making the system interact with the real world.

## Section 1: Your Mission

Your goal is to build a **detection-triggered alert system**. When the AI camera sees a specific object (like a person), it will trigger a physical buzzer to make a sound. You will also implement cooldown logic to prevent spam and create custom beep patterns for different objects.

### What Success Looks Like:
Your system will be silent until you place a 'person' in front of the camera, at which point it will emit two short beeps. If you show it a 'cell phone', it will emit a different pattern. It will only beep once per event, not continuously.

## Section 2: GPIO Basics

We need a way for the Raspberry Pi to control other electronics. We do this with the General Purpose Input/Output (GPIO) pins. We will use a library to make this simple.

In [ ]:
# Import necessary libraries
import sys, os, time
sys.path.append(os.path.abspath('../'))
from student_api import AIStudentAPI

# Reuse API object if this notebook section is re-run
if 'api' not in globals():
    api = AIStudentAPI()

if api.start_buzzer():
    print("Buzzer initialized on GPIO17. Let's test it.")
else:
    print("Failed to initialize buzzer on GPIO17.")

In [ ]:
# 2.1 - Simple on/off test
print("Beep!")
api.buzzer_on()      # Turn the buzzer on
time.sleep(0.5)      # Wait for half a second
api.buzzer_off()     # Turn the buzzer off

### EXERCISE: Make it beep three times
Using a `for` loop, make the buzzer beep 3 times with a short pause in between.

In [ ]:
for i in range(3):
    # Your on/off/sleep code here
    pass

### EXERCISE: Create an SOS pattern
The international signal for SOS is `... --- ...` (3 short, 3 long, 3 short). Create this pattern using the buzzer. Use a 0.2-second sleep for short beeps and a 0.6-second sleep for long beeps.

In [ ]:
# Your SOS code here

## Section 3: Detection Alerts

Now let's combine our live detection loop from Module 7 with our new buzzer control.

In [ ]:
# 3.1 - Full Setup (Camera + Buzzer)
import sys, os, cv2, time
from IPython.display import display, Image, clear_output
sys.path.append(os.path.abspath('../'))
from student_api import AIStudentAPI

# Reuse API object if this notebook section is re-run
if 'api' not in globals():
    api = AIStudentAPI()

if not api.is_running():
    if not api.start_camera():
        raise RuntimeError("Camera failed to start. Re-run this cell; the API will auto-recover stale camera sessions.")

if not api.start_buzzer():
    raise RuntimeError("Failed to initialize buzzer on GPIO17")

print("✅ System Initialized: Camera and Buzzer are live.")

In [ ]:
# 3.2 - Detection Loop with Alert (and Spam!)
try:
    while True:
        frame, detections = api.get_frame_and_detections()

        # Look for a person with high confidence
        for det in detections:
            if det['label'] == 'person' and det['confidence'] > 0.75:
                print("PERSON DETECTED! ALERT!")
                api.buzzer_on()
                time.sleep(0.1)
                api.buzzer_off()

            # Draw bounding boxes for visibility
            x, y, w, h = det['box']
            label = f"{det['label']}: {det['confidence']:.2f}"
            cv2.rectangle(frame, (x, y), (x + w, y + h), (4, 23, 115), 2)
            cv2.putText(frame, label, (x, max(15, y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (4, 23, 115), 2)

        # Display live frame below the cell
        _, img_encoded = cv2.imencode('.jpeg', frame)
        clear_output(wait=True)
        display(Image(data=img_encoded.tobytes()))

        time.sleep(0.1)  # Simulate camera frame rate

except KeyboardInterrupt:
    print("\nStream stopped.")

### EXERCISE: Add a Cooldown
The code above is very noisy! Modify the loop to only beep once every 5 seconds, even if a person is detected continuously. You will need a variable to keep track of the `last_beep_time`.

In [ ]:
last_beep_time = 0
COOLDOWN_SECONDS = 5

try:
    while True:
        _, detections = api.get_frame_and_detections()

        for det in detections:
            # TODO: add cooldown check and person detection condition
            # TODO: trigger double-beep and update last_beep_time
            pass

        time.sleep(0.1)

except KeyboardInterrupt:
    print("\nStream stopped.")



## Section 4: Custom Patterns

A single alert is good, but custom alerts for different objects are better.

In [ ]:
# 4.1 - Beep Pattern Dictionary
# Maps an object label to a beep pattern (on_time, off_time, number_of_beeps)
BEEP_PATTERNS = {
    'person': (0.1, 0.1, 2),     # Two short beeps
    'car': (0.5, 0.2, 1),       # One long beep
    'cell phone': (0.05, 0.05, 3) # Three very short beeps
}

### EXERCISE: Implement Custom Patterns
Modify your cooldown loop to use the `BEEP_PATTERNS` dictionary. When an object from the dictionary is detected, it should play the corresponding beep pattern.

In [ ]:
last_beep_time = 0
COOLDOWN_SECONDS = 5

try:
    while True:
        _, detections = api.get_frame_and_detections()

        # TODO: only process detections when cooldown has passed
        # TODO: check label in BEEP_PATTERNS and confidence > 0.7
        # TODO: play matched pattern, update cooldown timer, break

        time.sleep(0.1)

except KeyboardInterrupt:
    print("\nStream stopped.")



## Section 5: Knowledge Check

### EXERCISE: Fill in the Blanks
Complete the code for a full detection loop that includes the live video feed (from Module 7) and your custom beep pattern logic.

In [ ]:
# Your full code here, combining video and audio alerts

### EXERCISE: Short Answer
Why is cooldown logic essential for creating a usable alert system instead of just an annoying one?

// Your answer here

---
## Congratulations! You've completed Module 8.

In [ ]:
# Cleanly stop the API and release hardware
if 'api' in locals():
    api.shutdown_hardware(stop_camera=True, stop_buzzer=True)
    print("Camera and buzzer released.")